In [6]:
import numpy as np
import pandas as pd
import salishsea_tools.river_202108 as rivers
import xarray as xr
import matplotlib.pyplot as plt

In [4]:
bathy = xr.open_dataset('/home/sallen/MEOPAR/grid/bathymetry_202108.nc')

mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
# the mask is 1 where there is water, we want the opposite.  The meshmask has an extra dimension, hence the [0]
tmask = 1 - mesh.tmask[0]
area = ((1-tmask) * mesh.e1t[0] * mesh.e2t[0]).rename({'z':'depth', 'y':'gridY', 'x':'gridX'})[0]

In [32]:
rlist_dict = {'test': [['fraser', 'Fraser'], ['skagit', 'Skagit1']],
              'fraser': [['fraser', 'Fraser']],
              'Cassidy': [['fraser', 'Nooksack'], ['skagit', 'Skagit1'], ['skagit', 'SnohomishAllenQuilceda'], ['puget', 'NisquallyMcAllister'], \
                          ['jdf', 'Elwha'], ['evi_s', 'Cowichan1'], ['evi_s', 'Nanaimo1'], ['evi_s', 'Puntledge'], ['evi_n', 'SalmonSayward'], ['bute', 'Homathko'], \
                            ['howe', 'Squamish']],
              'Cassidy_all': [['fraser', 'Fraser'], ['fraser', 'Nooksack'], ['skagit', 'Skagit1'], ['skagit', 'SnohomishAllenQuilceda'], ['puget', 'NisquallyMcAllister'], \
                          ['jdf', 'Elwha'], ['evi_s', 'Cowichan1'], ['evi_s', 'Nanaimo1'], ['evi_s', 'Puntledge'], ['evi_n', 'SalmonSayward'], ['bute', 'Homathko'], \
                            ['howe', 'Squamish']]}

rho = 1026

In [15]:
rlist_dict['Cassidy_all']

[['fraser', 'Fraser'],
 ['fraser', 'Nooksack'],
 ['skagit', 'Skagit1'],
 ['skagit', 'SnohomishAllenQuilceda'],
 ['puget', 'NisquallyMcAllister'],
 ['jdf', 'Elwha'],
 ['evi_s', 'Cowichan1'],
 ['evi_s', 'Nanaimo1'],
 ['evi_s', 'Puntledge'],
 ['evi_n', 'SalmonSayward'],
 ['bute', 'Homathko'],
 ['howe', 'Squamish']]

In [23]:
def river_bounds(river):

    w_shed = river[0]
    r_call = river[1]

    y = rivers.prop_dict[w_shed][r_call]['i']  # model grid Y-axis
    x = rivers.prop_dict[w_shed][r_call]['j']  # model grid X-axis
    dy = rivers.prop_dict[w_shed][r_call]['di']  # the number of boxes in Y
    dx = rivers.prop_dict[w_shed][r_call]['dj']  # the number of boxes in X

    return y, dy, x, dx

In [17]:
river_dict = {'fraser_river': 'Fraser [kg/m2/s]', 'nooksack_river': 'Nooksack [kg/m2/s]', 'skagit_river': 'Skagit1 [kg/m2/s]', 'snohomish_river': 'SnohomishAllenQuilceda [kg/m2/s]', \
              'nisqually_river': 'NisquallyMcAllister [kg/m2/s]', 'elwha_river': 'Elwha [kg/m2/s]', 'cowichan_river': 'Cowichan1 [kg/m2/s]', \
              'nanaimo_river': 'Nanaimo1 [kg/m2/s]', 'puntledge_river': 'Puntledge [kg/m2/s]', 'salmon_river': 'SalmonSayward [kg/m2/s]', 'homathko_river': 'Homathko [kg/m2/s]', \
              'squamish_river': 'Squamish [kg/m2/s]'}

river_flows_202108 = pd.read_csv('/ocean/cdonaldson/MOAD/analysis-cassidy/notebooks/river_dye_trace/river_dailies_to_ts_all12_20190101_20231231.csv')
river_flows_202108 = river_flows_202108.loc[:, river_flows_202108.columns != 'Unnamed: 0']

river_flows_202108['date'] = pd.to_datetime(river_flows_202108['date'])
river_flows_202108['month'] = river_flows_202108['date'].dt.month
river_flows_202108['dayofyear'] = river_flows_202108['date'].dt.dayofyear
river_flows_202108

# river_flows_202108.loc[:, (river_flows_202108.columns != 'Unnamed: 0') & (river_flows_202108.columns != 'date') & (river_flows_202108.columns != 'month') & (river_flows_202108.columns != 'dayofyear')].plot()
# plt.plot(river_flows_202108['date'], river_flows_202108['Skagit1 [kg/m2/s]'].values)

,filename,date,Fraser [kg/m2/s],Nooksack [kg/m2/s],Skagit1 [kg/m2/s],SnohomishAllenQuilceda [kg/m2/s],NisquallyMcAllister [kg/m2/s],Elwha [kg/m2/s],Cowichan1 [kg/m2/s],Nanaimo1 [kg/m2/s],Puntledge [kg/m2/s],SalmonSayward [kg/m2/s],Homathko [kg/m2/s],Squamish [kg/m2/s],month,dayofyear
0,R202108Dailies_y2019m01d01.nc,2019-01-01,7.023721,0.087709,1.140777,1.594945,0.463038,0.271275,0.217430,0.188181,0.285031,0.528949,0.379871,0.708603,1,1
1,R202108Dailies_y2019m01d02.nc,2019-01-02,6.626748,0.069056,1.011028,1.413540,0.414510,0.254142,0.185309,0.160382,0.242924,0.477270,0.387039,0.584404,1,2
2,R202108Dailies_y2019m01d03.nc,2019-01-03,14.235843,0.506637,1.235469,1.727335,0.428012,3.603672,2.804349,2.427107,3.676248,1.927321,0.414514,0.619568,1,3
3,R202108Dailies_y2019m01d04.nc,2019-01-04,28.828118,1.324923,2.442424,3.414805,0.470989,3.175343,2.260775,1.956655,2.963671,3.228415,0.428251,1.216627,1,4
4,R202108Dailies_y2019m01d05.nc,2019-01-05,11.926503,0.307061,1.998030,2.793488,0.449749,1.142210,1.089619,0.943043,1.428392,1.708445,0.428848,1.215089,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,R202108Dailies_y2023m12d27.nc,2023-12-27,7.328564,0.065308,0.757089,1.058502,0.416669,0.376868,0.442156,0.382677,0.579626,1.436139,0.706493,1.692242,12,361
1822,R202108Dailies_y2023m12d28.nc,2023-12-28,6.941352,0.050072,0.726601,1.015876,0.342791,0.378492,0.731046,0.632705,0.958335,3.179205,0.940988,2.436159,12,362
1823,R202108Dailies_y2023m12d29.nc,2023-12-29,6.839268,0.043652,0.711089,0.994188,0.338203,0.421208,0.586788,0.507853,0.769225,2.199290,0.878170,2.343019,12,363
1824,R202108Dailies_y2023m12d30.nc,2023-12-30,6.847904,0.044195,0.656837,0.918338,0.337310,0.325631,0.389038,0.336705,0.509994,1.838149,0.986842,2.331040,12,364


In [27]:
np.array(area)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
for index in range(len(rlist_dict['Cassidy_all'])):
    # print(rlist_dict['Cassidy_all'][index])
    y, dy, x, dx = river_bounds(rlist_dict['Cassidy_all'][index])
    total_river_area = np.array(area)[y:y+dy, x:x+dx].sum()
    print(total_river_area)

211811.6058104098
439274.38587050105
223221.63871728705
225720.15728485544
227378.60781394705
223277.7046937642
219039.03381489814
431603.83224928624
212659.233210341
214279.48202068591
586760.7113249985
421132.61646910466


In [36]:
test_area = np.array([1, 1, 1.1])
test_fw = np.array([1, 1, 1])

sums_method = test_area.sum() * test_fw.sum()
dot_method = (test_area * test_fw).sum()

In [37]:
sums_method

9.3

In [38]:
dot_method

3.1